In [1]:
import time
import os
import math
import tushare as ts
from datetime import datetime
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from IPython.display import clear_output

sys.path.append("../utils/")
import date_util
import token_util
import enter_util
import stock_util

### 提取数据

In [2]:
trade_date = "20190830"
delta_price = 30.0

In [3]:
# stock_list = stock_util.get_stock_list(trade_date, delta_price)

In [4]:
stock_list = pd.read_csv("../data_pulled/stock_{}.csv".format(trade_date))
stock_list.head()

,ts_code,name,price
0,000001.SZ,平安银行,14.145
1,000002.SZ,万科A,25.885
2,000004.SZ,国农科技,20.090
3,000005.SZ,世纪星源,3.135
4,000006.SZ,深振业A,5.360


### 选出符合入场点的股票

In [5]:
# 选股

trade_date = '20190813'

columns = ["stock_code", "name", "pct_chg_short", "his_macd_discount", "amount_mean_20", "amount_discount_mean"]
raw_choose_list = pd.DataFrame(columns = columns)

ROOT_PATH = os.path.join("..", "data_pulled")
ROOT_PATH = os.path.join(ROOT_PATH, "day")
stock_csv_list = os.listdir(ROOT_PATH)


for i in range(len(stock_list)):
    
    stock_code = stock_list.iloc[i]["ts_code"]
    name = stock_list.iloc[i]["name"]
    
    if stock_code + ".csv" == stock_csv_list[i+1]:
        cur_detail = pd.read_csv(os.path.join(ROOT_PATH, stock_csv_list[i+1]))
        cur_index = cur_detail[cur_detail["trade_date"] == int(trade_date)].index.tolist()
        
        # 若此股票此时还未上市，则跳过
        if cur_index == []:
            continue
        else:
            cur_index = cur_index[0]
        
        cur_macd = cur_detail.iloc[cur_index]["macd"]
        last_macd =  cur_detail.iloc[cur_index-1]["macd"]
        cur_dif = cur_detail.iloc[cur_index]["diff"]
        cur_dea = cur_detail.iloc[cur_index]["dea"]
        pos_dif_mean = cur_detail["diff"][cur_detail["diff"] > 0].mean()
        pos_dea_mean = cur_detail["dea"][cur_detail["dea"] > 0].mean()
        
        # (最近3天涨跌幅).sum()
        ptc_chg_short = cur_detail["pct_chg"][cur_index-2:cur_index+1].sum()
        
        # 历史数据(MACD>0)天数 / 总的天数
        data_macd = cur_detail["macd"][:cur_index+1]
        his_macd_discount = len(data_macd[data_macd > 0]) / len(data_macd)
        
        # 近一个月成交量均值 / 近两个月成交量均值
        amount_mean_20 = cur_detail["amount"][cur_index - 20:cur_index].mean()
        amount_mean_40 = cur_detail["amount"][cur_index - 40:cur_index].mean()
        amount_discount_mean = amount_mean_20 / amount_mean_40
        
        # 判断 macd：今天 > 0, 昨天 < 0
        if cur_macd > 0 and last_macd < 0:
            # 判断今天的 dif 是否比 dea大
            if cur_dif - cur_dea > 0.001:
                # 判断 dif 和 dea 是否大于0，且小于正dif/dea的平均值
                if 0 < cur_dif < pos_dif_mean and 0 < cur_dea < pos_dea_mean:
                    list_tmp = pd.DataFrame([[stock_code, name, ptc_chg_short, his_macd_discount,
                                              amount_mean_20, amount_discount_mean]], columns=columns)
                    raw_choose_list = raw_choose_list.append(list_tmp)
        
    if i % 100 == 0:
        clear_output(True)
        print(i)
        print(raw_choose_list)

2600
  stock_code  name  pct_chg_short  his_macd_discount  amount_mean_20  \
0  000632.SZ  三木集团         4.0280           0.531036     20870.95435   
0  000823.SZ  超声电子         8.3256           0.519635    298526.66000   
0  002207.SZ  ST准油         5.0271           0.539091      5999.65705   
0  002216.SZ  三全食品         0.3699           0.521127     40183.95945   
0  002380.SZ  科远智慧         4.0964           0.514147     34046.11985   
0  601138.SH  工业富联         9.0510           0.577855    531231.20320   
0  601798.SH  ST蓝科         5.9515           0.556717      5345.79760   
0  603289.SH  泰瑞机器         2.1773           0.569794     44626.75565   
0  603328.SH  依顿电子        10.0255           0.542903    125764.87790   

   amount_discount_mean  
0              0.843801  
0              1.148239  
0              0.783564  
0              0.732591  
0              1.081248  
0              0.901000  
0              0.848848  
0              1.235318  
0              1.278268  


In [6]:
# 数据归一化

choose_list = raw_choose_list

for column in choose_list.columns[2:]:
    c_data = choose_list[column]
    choose_list[column] = (c_data-c_data.mean()) / c_data.std()

# 排序

def get_rank_factor(arr):
    pct_chg_short = arr["pct_chg_short"] * 0.5
    his_macd_discount = arr["his_macd_discount"]
    amount_mean_20 = arr["amount_mean_20"] * 0.5
    amount_discount_mean = arr["amount_discount_mean"] * 0.5
    return pct_chg_short+his_macd_discount+amount_mean_20+amount_discount_mean

choose_list["rank_factor"] = choose_list.apply(get_rank_factor, axis = 1)
choose_list = choose_list.sort_values(by="rank_factor", ascending= False).reset_index(drop=True)

In [7]:
choose_list

,stock_code,name,pct_chg_short,his_macd_discount,amount_mean_20,amount_discount_mean,rank_factor
0,601138.SH,工业富联,1.118062,1.609349,2.280794,-0.404285,3.106634
1,603328.SH,依顿电子,1.420653,0.067737,0.015696,1.441059,1.506441
2,603289.SH,泰瑞机器,-1.016281,1.253820,-0.437574,1.230975,1.142380
3,000823.SZ,超声电子,0.892819,-0.958535,0.980813,0.805044,0.380804
4,601798.SH,ST蓝科,0.155641,0.677040,-0.657013,-0.659374,0.096667
5,002207.SZ,ST准油,-0.131393,-0.100385,-0.653360,-0.978701,-0.982113
6,000632.SZ,三木集团,-0.441623,-0.455669,-0.570283,-0.684060,-1.303652
7,002380.SZ,科远智慧,-0.420384,-1.200611,-0.496681,0.477370,-1.420459
8,002216.SZ,三全食品,-1.577494,-0.892745,-0.462393,-1.228028,-2.526703


# TEST